# Imports

In [ ]:
# Get helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
!wget https://raw.githubusercontent.com/daoterog/Solid_Domestic_Waste_Image_Classification/main/helper_functions/file_management.py

In [ ]:
# Data Manipulation
import pandas as pd
import numpy as np

# Visualizations
import matplotlib.pyplot as plt

# TensorFlow
import tensorflow as tf
from tensorflow.keras import layers, regularizers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Evaluation
from sklearn.metrics import classification_report

# Helper functions
from helper_functions import (create_tensorboard_callback, plot_loss_curves, 
                              unzip_data, compare_historys, walk_through_dir,
                              make_confusion_matrix)
from file_management import bring_data, split_images

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

# Set seed for reproducibility
tf.random.set_seed(42)

# Load the Data

In [ ]:
path_list = ['/content/drive/MyDrive/PI2/data/cardboard.zip',
            '/content/drive/MyDrive/PI2/data/metal.zip']

# Unzip the data
bring_data(path_list=path_list)

# Split the dataset into train and test subsets
split_images(train_size=0.7, test_proportion=0.5)

# How many images/classes are there?
walk_through_dir("data")

In [ ]:
# Image paths
train_dir = '/content/data/train'
validation_dir = '/content/data/validation'
test_dir =  '/content/data/test'

IMG_SIZE = (224,224)
BATCH_SIZE = 32

# Load in the data
train_data = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                         label_mode='categorical',
                                                         image_size=IMG_SIZE,
                                                         batch_size=BATCH_SIZE,
                                                         shuffle=False)

validation_data = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                         label_mode='categorical',
                                                         image_size=IMG_SIZE,
                                                         batch_size=BATCH_SIZE,
                                                         shuffle=False)

test_data = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                         label_mode='categorical',
                                                         image_size=IMG_SIZE,
                                                         batch_size=BATCH_SIZE,
                                                         shuffle=False)

# Define Early Stopping Callback


In [ ]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                 patience=10)

# CNN Model

In [ ]:
# Create a Data Augmentation Layer
data_augmentation = Sequential([
    # preprocessing.RandomCrop(height=0.2,width=0.3), # Don't works for some reason
    preprocessing.RandomFlip(mode='horizontal'),
    preprocessing.RandomFlip(mode='vertical'),
    # preprocessing.RandomHeight(0.2),
    preprocessing.RandomRotation(0.3),
    preprocessing.RandomTranslation(0.3,0.3),
    # preprocessing.RandomWidth(0.2),
    # preprocessing.RandomZoom(0.2),
    preprocessing.RandomContrast(0.3),
    # layers.Rescaling(255.),
    layers.Resizing(224,224)
], name='data_augmentation_layer')

In [ ]:
cnn_data_augmentation = Sequential([
    data_augmentation,
    layers.Conv2D(32, 4, activation='relu', padding='same', 
                  kernel_regularizer=regularizers.l2(0.05)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Conv2D(20, 3, activation='relu', padding='same', 
                  kernel_regularizer=regularizers.l2(0.05)),
    layers.Conv2D(15, 3, activation='relu', padding='same', 
                  kernel_regularizer=regularizers.l2(0.05)),
    layers.Conv2D(12, 3, activation='relu', padding='same', 
                  kernel_regularizer=regularizers.l2(0.05)),
    layers.MaxPool2D(),
    layers.Conv2D(10, 3, activation='relu', padding='same', 
                  kernel_regularizer=regularizers.l2(0.05)),
    layers.Flatten(),
    layers.Dense(2, activation='softmax', name='output_layer')
])

# Compile the model
cnn_data_augmentation.compile(loss='binary_crossentropy',
                            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                            metrics=['accuracy'])

# Fit the model
cnn_data_augmentation_history = cnn_data_augmentation.fit(train_data,
                                                          epochs=50,
                                                          steps_per_epoch=len(train_data),
                                                          validation_data=validation_data,
                                                          batch_size=BATCH_SIZE*8,
                                                          validation_steps=len(validation_data),
                                                          callbacks=[create_tensorboard_callback(
                                                              dir_name='research_practice',
                                                              experiment_name='CNN'
                                                          ), earlystopping])

In [ ]:
# Assemble model
cnn_data_augmentation.summary()

In [ ]:
# Evaluate Model
cnn_data_augmentation_results = cnn_data_augmentation.evaluate(test_data)
cnn_data_augmentation_results

In [ ]:
plot_loss_curves(cnn_data_augmentation_history)

# Feature Extraction Model

In [ ]:
# Create Feature Extraction Model
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Define inputs
inputs = layers.Input(shape=IMG_SIZE + (3,), name='input_layer')
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(len(train_data.class_names), activation='softmax', 
                       name='output_layer')(x)

# Build Model
model = tf.keras.Model(inputs, outputs)

# Get summary
model.summary()

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

# Fit it
model_feature_vector_data_aug_history = model.fit(
    train_data, epochs=50, validation_data=validation_data, 
    callbacks=[earlystopping, create_tensorboard_callback(
        dir_name='research_practice',
        experiment_name='feature_extraction_model'
    )]
)

In [ ]:
# Evaluate Model
results_model_feature_vector_data_aug = model.evaluate(test_data)
results_model_feature_vector_data_aug

In [ ]:
plot_loss_curves(model_feature_vector_data_aug_history)

# Fine-Tuning Model

In [ ]:
# Let's unfreeze some layers of the base_model
NUM_UNFROZEN_LAYERS = 5

base_model.trainale = True
for layer in model.layers[2].layers[:-NUM_UNFROZEN_LAYERS]:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(lr=0.0001),
                metrics=['accuracy'])

# Fit it
fine_tuning_epochs = 10

model_fine_tuned_history = model.fit(
    train_data, epochs=50, validation_data=validation_data, 
    initial_epoch=model_feature_vector_data_aug_history.epoch[-1],
    callbacks=[earlystopping, create_tensorboard_callback(
        dir_name='research_practice',
        experiment_name='fine_tuning_model'
    )]
)

In [ ]:
# Evaluate the fine-tuned model
results_model_fine_tuned = model.evaluate(test_data)
results_model_fine_tuned

In [ ]:
# Compare history
compare_historys(original_history=model_feature_vector_data_aug_history,
                new_history=model_fine_tuned_history,
                initial_epochs=5)

# Upload to TensorBoard Hub

In [ ]:
!tensorboard dev list

In [ ]:
!tensorboard dev upload --logdir /content/research_practice --name "Research Practice Model Experiments" --one_shot --description "Here are stored the results of the final models resulting from my research practice II. You can check the notebook where this was generated in https://github.com/daoterog/Solid_Domestic_Waste_Image_Classification" \
    

# Evaluating the performance across classes

In [ ]:
# Make prediction with the model
pred_probs = cnn_data_augmentation.predict(test_data, verbose=1)

In [ ]:
# Make prediction with the model
pred_probs = model.predict(test_data, verbose=1)

In [ ]:
# Get image labels
y_labels = []
for _, label in test_data.unbatch():
    y_labels.append(label.numpy().argmax())

# Get predicted labels
y_pred = pred_probs.argmax(axis = 1)

# Get class names
class_names = test_data.class_names

In [ ]:
# Make confusion matrix with predicted labels
make_confusion_matrix(y_true=y_labels,
                      y_pred=y_pred,
                      classes=class_names)

In [ ]:
classification_report_dict = classification_report(y_true=y_labels,
                                                   y_pred=y_pred,
                                                   output_dict=True)

# Create empty dictionary
class_f1_scores = {}

# Loop through the classification report items
for key, value in classification_report_dict.items():
    if key == 'accuracy':
        break
    else:
        class_f1_scores[class_names[int(key)]] = value['f1-score']

class_f1_scores

In [ ]:
f1_scores = pd.DataFrame({"class_name": list(class_f1_scores.keys()),
                          "f1-score": list(class_f1_scores.values())})\
                          .sort_values("f1-score", ascending=False)

f1_scores

In [ ]:
def f1_scores_barplot(f1_scores, figsize=(10, 10)):
    fig, ax = plt.subplots(figsize=figsize)
    scores = ax.barh(range(len(f1_scores)), f1_scores["f1-score"].values)
    ax.set_yticks(range(len(f1_scores)))
    ax.set_yticklabels(list(f1_scores["class_name"]))
    ax.set_xlabel("f1-score")
    ax.set_title("F1-Scores for each Class")
    ax.invert_yaxis(); # reverse the order

f1_scores_barplot(f1_scores, figsize=(5,4))

# Find the most wrong predictions

In [ ]:
# Get the filenames of our test data
filepaths = []
for filepath in test_data.list_files('/content/data/test/*/*.jpg',
                                     shuffle=False):
    filepaths.append(filepath.numpy())

In [ ]:
# Create DataFrame
pred_df = pd.DataFrame({
    'img_path': filepaths,
    'y_true': y_labels,
    'y_pred': y_pred,
    'pred_prob': pred_probs.max(axis=1),
    'y_true_classname': [class_names[y] for y in y_labels],
    'y_pred_classname': [class_names[y] for y in y_pred]
})

# Add column that indicates wether the prediction was right
pred_df['pred_correct'] = pred_df.y_true == pred_df.y_pred

pred_df.head()

In [ ]:
# Get wrong predictions and sort them by their probability
wrong_preds = pred_df[~pred_df.pred_correct].sort_values(by='pred_prob',
                                                            ascending=False)
wrong_preds

In [ ]:
def load_and_prep_image(filename, img_shape=224, scale=True):
    """
    Reads in an image from filename, turns it into a tensor and reshapes into
    (224, 224, 3).

    Parameters
    ----------
    filename (str): string filename of target image
    img_shape (int): size to resize target image to, default 224
    scale (bool): whether to scale pixel values to range(0, 1), default True
    """
    # Read in the image
    img = tf.io.read_file(filename)
    # Decode it into a tensor
    img = tf.io.decode_image(img)
    # Resize the image
    img = tf.image.resize(img, [img_shape, img_shape])
    if scale:
        # Rescale the image (get all values between 0 and 1)
        return img/255.
    else:
        return img

In [ ]:
for row in wrong_preds.itertuples():
    _, img_path, _, _, pred_prob, true_cn, pred_cn, _ = row

    img = load_and_prep_image(img_path, scale=True)
    plt.imshow(img)
    plt.title(f"actual: {true_cn}, pred: {pred_cn} \nprob: {pred_prob:.2f}")
    plt.axis(False)
    plt.show()

In [ ]:
import sklearn
from sklearn.metrics import roc_auc_score

roc_auc_score(y_labels,y_pred)